In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/END

/content/drive/MyDrive/END


In [ ]:
# !wget http://www.comp.nus.edu.sg/~nlp/sw/sm_norm_mt.tar.gz

--2021-01-06 21:30:27--  http://www.comp.nus.edu.sg/~nlp/sw/sm_norm_mt.tar.gz
Resolving www.comp.nus.edu.sg (www.comp.nus.edu.sg)... 45.60.35.225
Connecting to www.comp.nus.edu.sg (www.comp.nus.edu.sg)|45.60.35.225|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.comp.nus.edu.sg/~nlp/sw/sm_norm_mt.tar.gz [following]
--2021-01-06 21:30:27--  https://www.comp.nus.edu.sg/~nlp/sw/sm_norm_mt.tar.gz
Connecting to www.comp.nus.edu.sg (www.comp.nus.edu.sg)|45.60.35.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180034 (176K) [application/x-gzip]
Saving to: ‘sm_norm_mt.tar.gz’

sm_norm_mt.tar.gz   100%[===================>] 175.81K  1.03MB/s    in 0.2s    

2021-01-06 21:30:27 (1.03 MB/s) - ‘sm_norm_mt.tar.gz’ saved [180034/180034]



In [ ]:
# !tar -xf /content/drive/MyDrive/END/sm_norm_mt.tar.gz

In [ ]:
# !rm -r /content/drive/MyDrive/END/sm_norm_mt.tar.gz

In [ ]:
%%bash
python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
!pip install google-trans-new

In [ ]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from tqdm.notebook import tqdm
import torchtext
import spacy
from spacy.lang.zh import Chinese
from torchtext.data import Field
import torch
import torch.nn as nn
from torchtext.data import BucketIterator
import torch.optim as optim
import time
import random
import math
from torchtext import data as data
from google_trans_new import google_translator

In [ ]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
text_data = pd.DataFrame(columns=['en', 'ch'])
with open('release/en2cn-2k.en2nen2cn') as file:
  while True:
      temp = file.readline()
      if len(temp) == 0:
          break
      
      en_text = file.readline().rstrip()
      ch_text = file.readline().rstrip()
      text_data = text_data.append({'en': en_text, 'ch': ch_text}, ignore_index=True)

In [ ]:
text_data

,en,ch
0,Do you want me to reserve seat for you or not?,你要我帮你预留坐位吗？
1,Yeap. You reaching? We ordered some Durian pas...,对。你要到了吗？我们已经点了一些榴莲糕点。你快点来。
2,They become more expensive already. Mine is li...,他们变得更贵了。我的是大概25。这么坏然后他们还比我以前做得少。
3,I'm Thai. What do you do?,我是泰国人。你做什么？
4,Hi! How did your week go? Haven't heard from y...,嗨！你这周过的怎么样？好长时间没听到你的消息了。一切顺利吗？
...,...,...
1995,Hmm. I think I usually book on weekends. It de...,嗯。我想我通常在周末预订。也不一定了。
1996,Can you ask them whether they have for any sms...,可以问问他们有简讯可以给我吗？现在我只有比100多一点点，还不够。你们都在线上吗？当你们都休...
1997,We are near Coca already.,我们离Coca很近了。
1998,Hall eleven. Got lectures. And forget about co...,11号宿舍。有讲课了。还有忘了复数等。还有好像很多Wk-self的。


In [ ]:
spacy_en = spacy.load('en')
spacy_zh = spacy.blank('zh')

In [ ]:
def tokenize_en(text):
  return [token.text for token in spacy_en.tokenizer(text)]

def tokenize_zh(text):
  return [token.text for token in spacy_zh.tokenizer(text)]

In [ ]:
SRC = Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<eos>', lower= True)
TRG = Field(tokenize=tokenize_zh, init_token='<sos>', eos_token='<eos>', lower= True)

In [ ]:
fields = [('src', SRC), ('trg', TRG)]

In [ ]:
examples = [data.Example.fromlist([text_data.en[i],text_data.ch[i]], fields) for i in range(text_data.shape[0])]

In [ ]:
NUS_Dataset = data.Dataset(examples, fields)

In [ ]:
train, valid, test = NUS_Dataset.split(split_ratio=[0.7, 0.15, 0.15], random_state=random.seed(SEED))

In [ ]:
len(train), len(valid), len(test)

(1400, 300, 300)

In [ ]:
def get_example(data, example_number):
    print(f"English Sentence : \n\n{' '.join(data.examples[example_number].src)}")
    print(f"\nChinese Sentence : \n\n{' '.join(data.examples[example_number].trg)}")

In [ ]:
get_example(train, 0)

English Sentence : 

why do n't you want to go ? tired ? or lazy ? i told you today is the last day to reply right ?   you still have n't made up your mind ?

Chinese Sentence : 

为什么 你 不想 去 ？ 累 了 ？ 还是 太懒 了 ？ 我 通知 你 今天 是 答复 的 最后 一天 了 对 吗 ？ 你 还是 没有 下定决心 ？


In [ ]:
SRC.build_vocab(train, min_freq = 2)
TRG.build_vocab(train, min_freq = 2)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [ ]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train, valid, test), 
    batch_size = BATCH_SIZE, sort = False,
    device = device)

In [ ]:
len(SRC.vocab), len(TRG.vocab)

(1171, 1305)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 512
DEC_EMB_DIM = 512
HID_DIM = 768
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(1171, 512)
    (rnn): GRU(512, 768)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(1305, 512)
    (rnn): GRU(1280, 768)
    (fc_out): Linear(in_features=2048, out_features=1305, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 11,618,585 trainable parameters


In [ ]:
optimizer = optim.AdamW(model.parameters())
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 15
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'NUS_model1.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 2s
	Train Loss: 6.336 | Train PPL: 564.482
	 Val. Loss: 5.021 |  Val. PPL: 151.600
Epoch: 02 | Time: 0m 2s
	Train Loss: 5.167 | Train PPL: 175.362
	 Val. Loss: 4.730 |  Val. PPL: 113.312
Epoch: 03 | Time: 0m 2s
	Train Loss: 4.992 | Train PPL: 147.259
	 Val. Loss: 4.714 |  Val. PPL: 111.475
Epoch: 04 | Time: 0m 2s
	Train Loss: 4.929 | Train PPL: 138.252
	 Val. Loss: 4.663 |  Val. PPL: 105.914
Epoch: 05 | Time: 0m 2s
	Train Loss: 4.862 | Train PPL: 129.322
	 Val. Loss: 4.660 |  Val. PPL: 105.611
Epoch: 06 | Time: 0m 2s
	Train Loss: 4.810 | Train PPL: 122.700
	 Val. Loss: 4.676 |  Val. PPL: 107.381
Epoch: 07 | Time: 0m 2s
	Train Loss: 4.769 | Train PPL: 117.816
	 Val. Loss: 4.699 |  Val. PPL: 109.871
Epoch: 08 | Time: 0m 2s
	Train Loss: 4.705 | Train PPL: 110.551
	 Val. Loss: 4.734 |  Val. PPL: 113.727
Epoch: 09 | Time: 0m 2s
	Train Loss: 4.681 | Train PPL: 107.842
	 Val. Loss: 4.699 |  Val. PPL: 109.886
Epoch: 10 | Time: 0m 2s
	Train Loss: 4.577 | Train PPL:  97.224


In [ ]:
model.load_state_dict(torch.load('NUS_model1.pt'))

<All keys matched successfully>

In [ ]:
test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 4.669 | Test PPL: 106.543 |


In [ ]:
batch = next(iter(valid_iterator))
src = batch.src
trg = batch.trg
prediction = model(src, trg, 0)

In [ ]:
def show_predictions(src, trg, example_id):
  translator = google_translator()
  model_prediction = " ".join([TRG.vocab.itos[x] for x in torch.argmax(prediction[:, example_id, :], axis = 1).cpu()])

  print("English Source Sentence:\n")
  print(" ".join([SRC.vocab.itos[x] for x in src[:, example_id].cpu()]))
    

  print("\nActual Target Sentence: \n")
  print(" ".join([TRG.vocab.itos[x] for x in trg[:, example_id].cpu()]))

  print("\nPredicted Target Sentence: \n")
  print(model_prediction)
  

In [ ]:
show_predictions(src, trg, 1)

English Source Sentence:

<sos> okay , then when i reach , i will go and collect the tickets first . now i have nothing to do , so watching tv . <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

Actual Target Sentence: 

<sos> 好 ， 那 我 <unk> 的 时候 ， 我会 先去 <unk> 。 现在 <unk> 做 ， 在 看电视 。 <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

Predicted Target Sentence: 

<unk> 我 ， 我 <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> 。 <eos> 。 <eos> 。 <eos> 。 <eos> 。 <eos> 。 <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos